In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
from tensorflow.keras import layers

from Hypers import *
from Utils import *
from Preprocessing import *
from Generator import *
from Models import *


In [4]:
data_path = ''

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'train_behaviors.tsv')
valid_news_file = os.path.join(data_path, 'train', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'train', r'val_behaviors.tsv')
test_news_file = os.path.join(data_path, 'valid', r'news.tsv')
test_behaviors_file = os.path.join(data_path, 'valid', r'test_behaviors.tsv')

In [6]:
news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict = read_news([train_news_file,test_news_file])
news_title,news_vert,news_subvert,news_entity,news_content,news_origin_title=get_doc_input(news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict)
word_embedding_path = '/home/myz/model/recommendations/pre-trained-model/glove.840B.300d.txt'
title_word_embedding_matrix, have_word = load_matrix(word_embedding_path,word_dict)

news_info = np.concatenate([news_title,news_vert.reshape((-1,1))],axis=-1)


In [ ]:
vert_embedding_matrix = np.zeros((19,300),dtype='float32')
for k,v in category_dict.items():
    if v==9 or v==16 or v==17:
        continue
    vert_embedding_matrix[v]=title_word_embedding_matrix[word_dict[k]]
    
vert_embedding_matrix[9]=(title_word_embedding_matrix[word_dict['food']]+title_word_embedding_matrix[word_dict['and']]+title_word_embedding_matrix[word_dict['drink']])/3
vert_embedding_matrix[16]=(title_word_embedding_matrix[word_dict['middle']]+title_word_embedding_matrix[word_dict['east']])
vert_embedding_matrix[17]=(title_word_embedding_matrix[word_dict['north']]+title_word_embedding_matrix[word_dict['america']])

In [ ]:
train_session = read_train_clickhistory(news_index,data_path,'train/train_behaviors.tsv',filter_num)
train_user = parse_user(news_index,train_session)
train_sess, train_user_id, train_label = get_train_input(news_index,train_session)
train_generator = get_hir_train_generator(news_info,train_user['click'],train_user_id,train_sess,train_label,32)

test_session = read_test_clickhistory(news_index,data_path,'valid/test_behaviors.tsv',filter_num)
test_user = parse_user(news_index,test_session)
test_docids, test_userids, test_labels, test_bound = get_test_input(news_index,test_session)

model,inter_model = create_model(title_word_embedding_matrix,vert_embedding_matrix)
model.fit_generator(train_generator,epochs=8)

test_generator = get_test_generator(news_info,test_docids,test_userids,test_user['click'],32)
predicted_label = inter_model.predict_generator(test_generator,verbose=1)
AUC, MRR, nDCG5, nDCG10 = evaluate(predicted_label,test_labels,test_bound)